The goal of this notebook is to explore how markers can be used to guide the algorithm finding the optimal window within a marker range. This is also useful for looking at marker ranges across an entire graph

In [25]:
import numpy as np
import pandas as pd
from graph_setup import G
from marker_functions import mkr_calc_window
from calc_functions import windows_test
from marker_functions import create_subs
import matplotlib.pyplot as plt
import networkx as nx

In [15]:
marker_ranges = mkr_calc_window(G, "PVH-937", "PVH-941")
print(marker_ranges)

Calculating windows for marker range 0 - K
100%|██████████| 50/50 [00:18<00:00,  2.66trial/s, best loss: 1.4624999999999773]
Optimal windows: [874.0, 1078.5, 870.5, 1070.5]
Minimum error: 1.4624999999999773
Calculating windows for marker range K - L2
100%|██████████| 50/50 [00:21<00:00,  2.34trial/s, best loss: 1.2930555555555605]
Optimal windows: [1077.5, 1290.5, 1077.5, 1290.5]
Minimum error: 1.2930555555555605
Calculating windows for marker range L2 - inf
100%|██████████| 50/50 [01:12<00:00,  1.46s/trial, best loss: 71.14159090909092]
Optimal windows: [1294.0, 1985.5, 1290.5, 1617.5]
Minimum error: 71.14159090909092
      Label           Range PVH-937           Range PVH-941   Opt Win PVH-937  \
0     0 - K          (0, 1075.1807)          (0, 1073.5878)   (874.0, 1078.5)   
1    K - L2  (1075.1807, 1290.3693)  (1073.5878, 1290.3293)  (1077.5, 1290.5)   
2  L2 - inf        (1290.3693, inf)        (1290.3293, inf)  (1294.0, 1985.5)   

    Opt Win PVH-941      Error  
0   (870.5, 107

In [24]:
all_graph_markers = pd.DataFrame()

for node in G.nodes:
    mk = G.nodes[node]["markers"]
    graph_markers = pd.DataFrame(columns=["Label", node])
     


    labels = []
    prev_marker = "0"
    for marker in mk["Mkr"]:
        labels.append(f"{prev_marker} - {marker}")
        prev_marker = marker
    labels.append(f"{prev_marker} - inf")

    graph_markers["Label"] = labels

    sub = create_subs(mk["Depth"].tolist())
    graph_markers[node] = sub


    if all_graph_markers.empty:
        all_graph_markers = graph_markers
    else:
        all_graph_markers = pd.merge(all_graph_markers, graph_markers, on="Label", how="outer")

print(all_graph_markers)
    

             Label                 PVH-1007                 PVH-937  \
0            0 - C          (0, 1011.38385)           (0, 969.7597)   
1            C - K  (1011.38385, 1116.2677)   (969.7597, 1075.1807)   
2           K - L2   (1116.2677, 1333.9681)  (1075.1807, 1290.3693)   
3         L2 - inf         (1333.9681, inf)                     NaN   
4   L2 - Top-MEC-C                      NaN  (1290.3693, 1706.3507)   
5  Top-MEC-C - inf                      NaN        (1706.3507, inf)   
6            0 - K                      NaN                     NaN   

                  PVH-941                PVH-1108                PVH-1318  \
0                     NaN          (0, 1008.8733)          (0, 1007.1922)   
1                     NaN  (1008.8733, 1113.1971)  (1007.1922, 1111.3737)   
2  (1073.5878, 1290.3293)  (1113.1971, 1324.5449)  (1111.3737, 1327.1564)   
3        (1290.3293, inf)                     NaN                     NaN   
4                     NaN  (1324.5449, 1770.02

In [32]:
marker_graphs = {}
for _, row in all_graph_markers.iterrows():
    label = row["Label"]
    tempG = nx.Graph()
    
    # Add nodes and edges to the graph for the current row
    for node in G.nodes:
        if node in row.index and pd.notna(row[node]):
            tempG.add_node(node, **G.nodes[node])
            tempG.nodes[node]["data"] = G.nodes[node]["data"]
            tempG.nodes[node]["known_tops"] = G.nodes[node]["known_tops"]
            tempG.nodes[node]["pos"] = G.nodes[node]["pos"]

    marker_graphs[label] = tempG

# Example of how to access a specific marker graph
for label, graph in marker_graphs.items():
    print(f"Graph for label {label}:")
    print(graph.nodes)




Graph for label 0 - C:
['PVH-1007', 'PVH-937', 'PVH-1108', 'PVH-1318', 'PVH-1499']
Graph for label C - K:
['PVH-1007', 'PVH-937', 'PVH-1108', 'PVH-1318', 'PVH-1499']
Graph for label K - L2:
['PVH-1007', 'PVH-937', 'PVH-941', 'PVH-1108', 'PVH-1318', 'PVH-1499']
Graph for label L2 - inf:
['PVH-1007', 'PVH-941']
Graph for label L2 - Top-MEC-C:
['PVH-937', 'PVH-1108', 'PVH-1318', 'PVH-1499']
Graph for label Top-MEC-C - inf:
['PVH-937', 'PVH-1108', 'PVH-1318', 'PVH-1499']
Graph for label 0 - K:
['PVH-941']
